In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class ClassificationNetwork(nn.Module):
    def __init__(self, num_classes):
        super(ClassificationNetwork, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 5)
        self.conv2 = nn.Conv2d(64, 128, 5)
        self.conv3 = nn.Conv2d(128, 128, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(2048, 256)  # Adjust the size according to your input dimension
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [8]:
num_classes = 2  # Update this based on your dataset
model = ClassificationNetwork(num_classes=num_classes)


In [9]:
model_path = '/Users/mac/Documents/GitHub/MY-DMLAP-2024/python/05-intro-to-PyTorch-NN-trainings/ml/img_classfier_model_e5.pt'
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
model.eval()  # Set the model to evaluation mode


ClassificationNetwork(
  (conv1): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(128, 128, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=2048, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=2, bias=True)
)

In [12]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transformations
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize images to the same size as training
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize images as during training
])

# Load your unseen dataset
unseen_dataset = datasets.ImageFolder(root='/Users/mac/Documents/GitHub/MY-DMLAP-2024/python/05-intro-to-PyTorch-NN-trainings/ml/gallery-dl/pinterest/unforseen', transform=transform)

# Create the DataLoader for your unseen dataset
unseen_loader = DataLoader(unseen_dataset, batch_size=100, shuffle=False)


In [13]:
correct = 0
total = 0
with torch.no_grad():
    for images, labels in unseen_loader:
        images, labels = images.to('cpu'), labels.to('cpu')  # Adjust if using a GPU
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Accuracy on unseen data: {accuracy * 100:.2f}%')

Accuracy on unseen data: 91.75%
